<a href="https://colab.research.google.com/github/HuyenNguyenHelen/LING-5412/blob/main/Assignment_Feedforward_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import os
import re
import shutil
import tensorflow as tf
import string
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
import glob
print(tf.__version__)


2.6.0


# Part 1: IMDB sentiment analysis

## Loading the dataset

In [ ]:
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
data = tf.keras.utils.get_file ('aclImdb_v1',
                                url,
                                untar = True,
                                cache_dir = '.',
                                cache_subdir = '')
data_dir = os.path.join (os.path.dirname(data), 'aclImdb')
print(os.listdir(data_dir))

84140032/84125825 [==============================] - 6s 0us/step
['imdb.vocab', 'test', 'README', 'imdbEr.txt', 'train']


In [ ]:
train_dir = os.path.join (data_dir, 'train')
print(os.listdir(train_dir))

['neg', 'urls_neg.txt', 'urls_pos.txt', 'unsup', 'pos', 'urls_unsup.txt', 'unsupBow.feat', 'labeledBow.feat']


In [ ]:
# We only use files in the two folders: pos, and neg, so let's remove other files
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [ ]:
# Loading data from the directory
batch_size = 32
seed = 42
raw_train = tf.keras.utils.text_dataset_from_directory ('aclImdb/train',
                                                        batch_size =batch_size,
                                                        validation_split = 0.2,
                                                        subset = 'training',
                                                        seed = seed)
raw_val = tf.keras.utils.text_dataset_from_directory ('aclImdb/train',
                                                      batch_size = batch_size,
                                                      validation_split = 0.2,
                                                      subset = 'validation',
                                                      seed = seed)
raw_test = tf.keras.utils.text_dataset_from_directory ('aclImdb/test',
                                                       batch_size = batch_size)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


# Text representation

In [ ]:
def custom_preprocessing (text):
  lowercase = tf.strings.lower (text)
  stripped_html = tf.strings.regex_replace (lowercase,'<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), 
                                  '')
  
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(standardize = custom_preprocessing,
                                           max_tokens = max_features,
                                           output_mode = 'int',
                                           output_sequence_length = sequence_length)
# Extracting features for vectorizing using training set
train_text = raw_train.map (lambda x, y: x)
vectorize_layer.adapt(train_text)

# Defining a function for fitting vectorizer function/layer to vectorize text (review)
def fitting_vectorizer (text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer (text), label

# storing text batch and label batch
text_batch, label_batch = next(iter(raw_train))

## print an instance with vectorized review and label for observing
print ('REVIEW:', text_batch[0])
print('LABEL:', raw_train.class_names[label_batch[0]] )


REVIEW: tf.Tensor(b'Silent Night, Deadly Night 5 is the very last of the series, and like part 4, it\'s unrelated to the first three except by title and the fact that it\'s a Christmas-themed horror flick.<br /><br />Except to the oblivious, there\'s some obvious things going on here...Mickey Rooney plays a toymaker named Joe Petto and his creepy son\'s name is Pino. Ring a bell, anyone? Now, a little boy named Derek heard a knock at the door one evening, and opened it to find a present on the doorstep for him. Even though it said "don\'t open till Christmas", he begins to open it anyway but is stopped by his dad, who scolds him and sends him to bed, and opens the gift himself. Inside is a little red ball that sprouts Santa arms and a head, and proceeds to kill dad. Oops, maybe he should have left well-enough alone. Of course Derek is then traumatized by the incident since he watched it from the stairs, but he doesn\'t grow up to be some killer Santa, he just stops talking.<br /><br />

In [ ]:
# print an example of vectorized data
print ('Vocabulary size: ', len(vectorize_layer.get_vocabulary()))
for i in range (90, 100):
  print ('{} ------> {}'.format(i, vectorize_layer.get_vocabulary()[i]))

Vocabulary size:  10000
90 ------> made
91 ------> movies
92 ------> then
93 ------> them
94 ------> films
95 ------> way
96 ------> make
97 ------> any
98 ------> could
99 ------> too


In [ ]:
train = raw_train.map(fitting_vectorizer)
val = raw_val.map(fitting_vectorizer)
test = raw_test.map(fitting_vectorizer)


In [ ]:
# Configure the dataset for performance
autotune = tf.data.AUTOTUNE
train = train.cache().prefetch (buffer_size = autotune)
val = val.cache().prefetch (buffer_size = autotune)
test = test.cache().prefetch (buffer_size = autotune)

# Building a neural network classifier

## With different numbers of embedding dimentions

In [ ]:
# Creating the model
embedding_dim = [16, 28, 50, 100]
for n in embedding_dim:
  print ("========= embedding vectors'size= %s ============" %n)
  model = tf.keras.Sequential([layers.Embedding(max_features + 1, n),
                              layers.Dropout(0.2),
                              layers.GlobalAveragePooling1D(),
                              layers.Dropout(0.2),
                              layers.Dense(1)])
  print(model.summary())

  # configure the model uisng optimizer and loss function
  model.compile(loss = losses.BinaryCrossentropy(from_logits = True),
                optimizer = 'adam',
                metrics = tf.metrics.BinaryAccuracy(threshold = 0.0 )) ## Why threshold = 0.0??
  # training the model
  epochs = 10
  history = model.fit(train,
                      validation_data = val,
                      epochs = epochs)
  # testing the model
  loss, accuracy = model.evaluate(test)
  print('Testing performance:\n Loss: {} - Accuracy: {}'. format(loss, accuracy))

========= embedding vectors'size= 16 ============
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160016    
_________________________________________________________________
dropout (Dropout)            (None, None, 16)          0         
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 160,033
Trainable params: 160,033
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
625/625 [==============================] - 7s 10ms

## With different dropout 

In [ ]:
# Creating the model
embedding_dim = 16
dropouts = [0.0, 0.1, 0.2, 0.3]
for i in dropouts:
  print ("========= dropout = %s ============" %i)
  model = tf.keras.Sequential([layers.Embedding(max_features + 1,embedding_dim),
                              layers.Dropout(i),
                              layers.GlobalAveragePooling1D(),
                              layers.Dropout(0.2),
                              layers.Dense(1)])
  print(model.summary())

  # configure the model uisng optimizer and loss function
  model.compile(loss = losses.BinaryCrossentropy(from_logits = True),
                optimizer = 'adam',
                metrics = tf.metrics.BinaryAccuracy(threshold = 0.0 )) ## Why threshold = 0.0??
  # training the model
  epochs = 10
  history = model.fit(train,
                      validation_data = val,
                      epochs = epochs)
  # testing the model
  loss, accuracy = model.evaluate(test)
  print('Testing performance:\n Loss: {} - Accuracy: {}'. format(loss, accuracy))

========= dropout = 0.0 ============
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 16)          160016    
_________________________________________________________________
dropout_8 (Dropout)          (None, None, 16)          0         
_________________________________________________________________
global_average_pooling1d_4 ( (None, 16)                0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total params: 160,033
Trainable params: 160,033
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
625/625 [==============================] - 3s 3ms/step - loss

## Adding a Dense layer

In [ ]:
# Creating the model
embedding_dim = 16
dropout =  0.1
activations = ['relu', 'softmax', 'sigmoid']
for f in activations:
  print ("========= activation function = %s ============" %f)
  model = tf.keras.Sequential([layers.Embedding(max_features + 1,embedding_dim),
                              layers.Dropout(dropout),
                              layers.GlobalAveragePooling1D(),
                              layers.Dropout(dropout),
                              layers.Dense(32, activation= f),
                              layers.Dense(1)])
  print(model.summary())

  # configure the model uisng optimizer and loss function
  model.compile(loss = losses.BinaryCrossentropy(from_logits = True),
                optimizer = 'adam',
                metrics = tf.metrics.BinaryAccuracy(threshold = 0.0 )) ## Why threshold = 0.0??
  # training the model
  epochs = 10
  history = model.fit(train,
                      validation_data = val,
                      epochs = epochs)
  # testing the model
  loss, accuracy = model.evaluate(test)
  print('Testing performance:\n Loss: {} - Accuracy: {}'. format(loss, accuracy))

========= activation function = relu ============
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, None, 16)          160016    
_________________________________________________________________
dropout_18 (Dropout)         (None, None, 16)          0         
_________________________________________________________________
global_average_pooling1d_9 ( (None, 16)                0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 32)                544       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 33        
Total params: 160,593
Trainable params: 160,593
Non-trainable params: 

## With different Batch size

In [ ]:
# Loading data from the directory
batch_size = 64
seed = 42
raw_train = tf.keras.utils.text_dataset_from_directory ('aclImdb/train',
                                                        batch_size =batch_size,
                                                        validation_split = 0.2,
                                                        subset = 'training',
                                                        seed = seed)
raw_val = tf.keras.utils.text_dataset_from_directory ('aclImdb/train',
                                                      batch_size = batch_size,
                                                      validation_split = 0.2,
                                                      subset = 'validation',
                                                      seed = seed)
raw_test = tf.keras.utils.text_dataset_from_directory ('aclImdb/test',
                                                       batch_size = batch_size)

# storing text batch and label batch
text_batch, label_batch = next(iter(raw_train))

## print an instance with vectorized review and label for observing
print ('REVIEW:', text_batch[0])
print('LABEL:', raw_train.class_names[label_batch[0]] )


# Creating the model
embedding_dim = 16
dropout =  0.1
activation =  'softmax'

print ("======== activation function = {}, dropout = {}, batch size = {} ============".format(activation, dropout, batch_size ))
model = tf.keras.Sequential([layers.Embedding(max_features + 1,embedding_dim),
                            layers.Dropout(dropout),
                            layers.GlobalAveragePooling1D(),
                            layers.Dropout(dropout),
                            layers.Dense(32, activation= activation),
                            layers.Dense(1)])
print(model.summary())

# configure the model uisng optimizer and loss function
model.compile(loss = losses.BinaryCrossentropy(from_logits = True),
              optimizer = 'adam',
              metrics = tf.metrics.BinaryAccuracy(threshold = 0.0 )) ## Why threshold = 0.0??
# training the model
epochs = 10
history = model.fit(train,
                    validation_data = val,
                    epochs = epochs)
# testing the model
loss, accuracy = model.evaluate(test)
print('Testing performance:\n Loss: {} - Accuracy: {}'. format(loss, accuracy))

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
REVIEW: tf.Tensor(b"First of all, I liked very much the central idea of locating the '' intruders'', Others in the fragile Self, on various levels - mainly subconscious but sometimes more allegorical. In fact the intruders are omnipresent throughout the film : in the Swiss-French border where the pretagonist leads secluded life; in the his recurring daydream and nightmare; inside his ailing body after heart transplantation.... In the last half of the film, he becomes intruder himself, returning in ancient french colony in the hope of atoning for the past. <br /><br />The overall tone is bitter rather than pathetic, full of regrets and guilts, sense of failure being more or less dominant. This is a quite grim picture of an old age, ostensibly self-dependent but hopelessly void and lonely inside. The

## With different training algorithm
Here we mostly focus on adapting learning rate method

In [16]:
# configure the model uisng optimizer and loss function
optimizers = ['adagrad', 'rmsprop', 'adam']

print ("======== activation function = {}, dropout = {}, batch size = {} ============".format(activation, dropout, batch_size ))
for opt in optimizers:
  print( '========== optimizer = %s' %opt)
  model.compile(loss = losses.BinaryCrossentropy(from_logits = True),
                optimizer = opt,
                metrics = tf.metrics.BinaryAccuracy(threshold = 0.0 )) ## Why threshold = 0.0??
  # training the model
  epochs = 10
  history = model.fit(train,
                      validation_data = val,
                      epochs = epochs)
  # testing the model
  loss, accuracy = model.evaluate(test)
  print('Testing performance:\n Loss: {:.3f} - Accuracy: {:.3f}'. format(loss, accuracy))

======== activation function = softmax, dropout = 0.1, batch size = 64 ============
========== optimizer = adagrad
Epoch 1/10
625/625 [==============================] - 3s 4ms/step - loss: 0.1539 - binary_accuracy: 0.9535 - val_loss: 0.3054 - val_binary_accuracy: 0.8804
Epoch 2/10
625/625 [==============================] - 2s 4ms/step - loss: 0.1533 - binary_accuracy: 0.9543 - val_loss: 0.3055 - val_binary_accuracy: 0.8800
Epoch 3/10
625/625 [==============================] - 3s 4ms/step - loss: 0.1534 - binary_accuracy: 0.9546 - val_loss: 0.3056 - val_binary_accuracy: 0.8804
Epoch 4/10
625/625 [==============================] - 3s 4ms/step - loss: 0.1530 - binary_accuracy: 0.9543 - val_loss: 0.3056 - val_binary_accuracy: 0.8808
Epoch 5/10
625/625 [==============================] - 3s 4ms/step - loss: 0.1532 - binary_accuracy: 0.9551 - val_loss: 0.3057 - val_binary_accuracy: 0.8802
Epoch 6/10
625/625 [==============================] - 2s 4ms/step - loss: 0.1537 - binary_accuracy: 0.953

# Word Embedding

# Comparing with a Logistic Regression model

In [36]:
# loading the ds
pos, neg = [], []
def Getdata(dir):
  for folder in glob.glob (dir+'/*/*'):
    for file in glob.glob(folder+'/*'): 
      fo = open(file)
      doc = fo.read()
      if 'pos' in file:
        pos.extend(doc)

    print(doc)
Getdata(train_dir)

  

Don't be deceived as I was by the 'glowing' reviews quoted on the DVD box. "Wildly entertaining.", "a seriously scary freakout.", and the worst of all, "ON PAR WITH JAWS." This movie is none of the above.<br /><br />Normally I don't bother with writing bad reviews for films but I can't believe this one is resting at a comfortable 7 on IMDb. It doesn't deserve it.<br /><br />After a so-so opening daylight attack by a monster created by, what else, chemicals dumped by lazy scientists, this movie goes absolutely nowhere and it goes there sloooowly. Basically and improbably, a girl is snagged by the monster (I'll give them points for a good creature design but this ain't no WETA creation) and her semi-comical family spend an hour-and-a-half tracking her down...in the sewers surrounding the Han river. Their search lacks any suspense-again, someone called this on par with Jaws?-and by the time they find her you realize it was all pretty much pointless. Other than that, a big bulk of the movi

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



It's not just that this is a bad movie; it's not only that four of the "best" Mexican movie makers are in this film; and it's not only that the script is terrible. It's just that...this movie sucks...big time. This people are wasting money in terrible scripts. It's supposed to make a criticism about Mexican society but we're fed up with this kind of films. Is bad language supposed to be funny? I don't get it. Mexican cinema is in big trouble if this kind of movies are going to continue playing (and being written and produced).<br /><br />Please, don't think this kind of movies are well received in Mexico: We hate them and they don't reflect us.
I watch lots of scary movies (or at least they try to be) and this has to be the worst if not 2nd worst movie I have ever had to make myself try to sit through. I never knew the depths of Masacism until I rented this piece of moldy cheese covered in a used latex contraceptive. I am a fan of Julian Sans, but this is worse than I would hope for hi

KeyboardInterrupt: ignored

In [ ]:
# Sklearn Logistic Regression Model
from sklearn.linear_model import LogisticRegression

def printing_eval_scores (y_true, y_pred, report=''):
  accuracy = sklearn.metrics.accuracy_score(y_true, y_pred)
  precision = sklearn.metrics.precision_score(y_true, y_pred)
  recall = sklearn.metrics.recall_score(y_true, y_pred)
  f1 = sklearn.metrics.f1_score(y_true, y_pred)
  print('accuracy score: {:.3f}'.format(accuracy))
  print('precision score: {:.3f}'.format(precision))
  print('recall score: {:.3f}'.format(recall))
  print('F1 score: {:.3f}'.format(f1))
  if report is True:
    print(classification_report(y_true, y_pred))
  else:
    pass
  return accuracy, precision, recall, f1

X_train = text
# Vectorizing the documents
vectorizer = CountVectorizer(binary = True)
X_train = vectorizer.fit_transform(X_train.to_list())
X_test = vectorizer.transform(X_test.to_list())
print ('Shapes of X_train, y_train: ', X_train.shape, y_train.shape)
print ('Shapes of X_test, y_test: ', X_test.shape, y_test.shape)

sk_lr = LogisticRegression(solver='lbfgs', max_iter=150).fit(X_train, y_train )
y_predict = sk_lr.predict(X_test)

# Model performing
## on training set
print('Model performance on training set:')
printing_eval_scores (y_train, sk_lr.predict(X_train))

## on test set
print('\n===========================')
print('Model performance on test set:')
printing_eval_scores (y_test, y_predict, report = True)